 Deductive reasononig:is making observations from specific data points and then forming a general conclusion based on patterns.
 

In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report, roc_curve, auc
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import ConfusionMatrixDisplay
from typing import List
from typing import Tuple
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

XGBoostError: 
XGBoost Library (libxgboost.dylib) could not be loaded.
Likely causes:
  * OpenMP runtime is not installed
    - vcomp140.dll or libgomp-1.dll for Windows
    - libomp.dylib for Mac OSX
    - libgomp.so for Linux and other UNIX-like OSes
    Mac OSX users: Run `brew install libomp` to install OpenMP runtime.

  * You are running 32-bit Python on a 64-bit OS

Error message(s): ["dlopen(/opt/homebrew/Cellar/jupyterlab/4.2.5/libexec/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib, 0x0006): Library not loaded: @rpath/libomp.dylib\n  Referenced from: <F2F42313-BF4F-3B95-A853-AE1DE94D4C87> /opt/homebrew/Cellar/jupyterlab/4.2.5/libexec/lib/python3.12/site-packages/xgboost/lib/libxgboost.dylib\n  Reason: tried: '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/opt/homebrew/opt/libomp/lib/libomp.dylib' (no such file), '/System/Volumes/Preboot/Cryptexes/OS/opt/home

In [ ]:
df = pd.read_csv('processed.hungarian.csv')

In [ ]:
def preprocess_data(df: pd.DataFrame, column_names: List[str]) -> pd.DataFrame:

    df.columns = column_names

    df['ca'] = pd.to_numeric(df['ca'], errors='coerce')
    df['thal'] = pd.to_numeric(df['thal'], errors='coerce')

    df['ca'] = df['ca'].fillna(df['ca'].median())
    df['thal'] = df['thal'].fillna(df['thal'].median())

    df['num'] = np.where(df['num'] >= 2, 1, df['num'])

    return df
df = preprocess_data(df, column_names)
df.head()


In [ ]:
from typing import Tuple
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report

def train_and_evaluate_model(df: pd.DataFrame) -> Tuple[str, DecisionTreeClassifier]:
    X = df.drop('num', axis=1)
    y = df['num']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
    model = DecisionTreeClassifier(
        class_weight='balanced',
        max_features=10, 
        splitter='best',  
        max_depth=7, 
        random_state=42
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred)
    return report, model
report, model = train_and_evaluate_model(df)
print(report)

wrgbiugbr90er

In [ ]:
def train_and_evaluate_rf_model(df: pd.DataFrame) -> Tuple[float, RandomForestClassifier]:
    X = df.drop('num', axis=1)
    y = df['num']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    rf = RandomForestClassifier(
        n_estimators=300,         
        max_depth=12,            
        min_samples_split=3,      
        min_samples_leaf=1,       
        class_weight='balanced', 
        random_state=42
    )
    rf.fit(X_train, y_train)
    score = rf.score(X_test, y_test)
    ConfusionMatrixDisplay.from_estimator(rf, X_test, y_test)
    return score, rf
score, rf = train_and_evaluate_rf_model(df)

In [ ]:
recall


In [ ]:
def train_and_evaluate_xgb_model(df: pd.DataFrame) -> Tuple[XGBClassifier, pd.Series]:
    X = df.drop('num', axis=1)
    y = df['num']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
    xgb = XGBClassifier(
        n_estimators=300,          
        max_depth=10,
        gamma=1
    )
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)
    ConfusionMatrixDisplay.from_estimator(xgb, X_test, y_test)
    return xgb, y_pred
xgb, y_pred = train_and_evaluate_xgb_model(df)

In [ ]:
recall